# Qué entidades suben nuevos datos a GeoBolivia?

El monitor de datos geográficos que mantengo ha estado corriendo por 6 meses sin interrupciones. Ocasionalmente reviso los [diffs](https://github.com/mauforonda/geodatos/commits/) para ver qué datos actualizan entidades de gobierno. Creo que estas actualizaciones indican algunas preguntas que se hacen analistas en el gobierno, reportes que demandan, proyectos que deben justificar o fenómenos que monitorean.

GeoBolivia es un caso especial. Cuando una entidad sube un dataset a GeoBolivia asume que será público. En otros servidores geográficos, los analistas cambian datos con menos cuidado. GeoBolivia es más un commons, otros servidores son infraestructura de la entidad. Hace un año, la vicepresidencia estaba apunto de cerrar silenciosamente GeoBolivia. Tras una campaña de comunicación, logramos que se restablezca el servicio y probablemente asignen algo de presupuesto a su mantenimiento. Sin embargo, siguió pendiente la pregunta de si habían personas que encontraban valor práctico en este servicio. Existen personas que consultan estos datos? Existen funcionarios públicos que utilizan este servicio en lugar de invertir en recursos redundantes sobre los que puedan tener mayor control dentro de sus entidades? Existen alguien dentro del gobierno que crea en la visión de una infraestructura de datos compartida y abierta? 

En este notebook reviso el historial del catálogo de datos geográficos para identificar nuevos datasets que han sido subidos a GeoBolivia desde febrero de 2022.

In [110]:
import datetime as dt
from io import StringIO

import git
import pandas as pd
from IPython.display import Markdown, display

pd.options.display.max_colwidth = 400

In [162]:
display(Markdown('Son las `{}`'.format(dt.datetime.now(tz=dt.timezone(dt.timedelta(hours=-4))).isoformat())))

Son las `2022-08-04T12:11:35.254318-04:00`

- `data/geodatos` es la ubicación de una copia local de [github.com/mauforonda/geodatos](https://github.com/mauforonda/geodatos/)
- `geodatos.csv` es el catálogo de datos

In [22]:
repo = git.Repo("data/geodatos/")
filename = "geodatos.csv"

Cóm identifico nuevos datasets en GeoBolvia. 
- Recorro cada commit en el repositorio. 
- Comparo los datasets de GeoBolivia entre cada par consecutivo de commits donde haya habido al menos uno (a veces algunos servidores no responden y no reportan datos).
- Colecciono estos nuevos datasets con la estampa de tiempo de la fecha en que primero aparecen.

In [116]:
datasets = []
previous = None
previous_timestamp = None
for commit in repo.iter_commits(rev="master"):
    timestamp = dt.datetime.fromtimestamp(commit.committed_date)
    if filename in [e.name for e in commit.tree]:
        data = pd.read_csv(StringIO(commit.tree[filename].data_stream.read().decode()))
        geobolivia = data[data.sistema == "GeoBolivia"]
        if geobolivia.shape[0] > 0:
            if previous is not None:
                new = pd.concat([previous, geobolivia]).drop_duplicates(
                    subset=["nombre"], keep=False
                )
                datasets.extend(
                    [
                        {
                            "timestamp": previous_timestamp,
                            "entidad": row["nombre"].split(":")[0],
                            "titulo": row["titulo"],
                            "nombre": row["nombre"],
                        }
                        for i, row in new.iterrows()
                    ]
                )
            previous = geobolivia
            previous_timestamp = timestamp

In [118]:
datasets = pd.DataFrame(datasets)

Éstas son las entidades que suben más datos a GeoBolivia

In [125]:
porentidad = pd.concat(
    [
        datasets.entidad.value_counts(),
        datasets.entidad.value_counts(normalize=True) * 100,
    ],
    axis=1,
)
porentidad.columns = ["nuevos datasets", "%"]
porentidad

,nuevos datasets,%
GobMunicipal,166,66.666667
abt,15,6.024096
mdenerg,12,4.819277
nasa,9,3.614458
geoelectoral,8,3.212851
otros,5,2.008032
autoridadelectricidad,5,2.008032
inra,4,1.606426
universidades,4,1.606426
mapashistoricos,4,1.606426


Es un grupo pequeño respecto a todas las entidades que subieron datos en algún momento en el pasado.

In [128]:
display(
    Markdown(
        "Sólo {} de {} entidades registradas subieron nuevos datos desde febrero de 2022".format(
            len(datasets.entidad.unique()),
            len(
                pd.read_csv("data/geodatos/geodatos.csv")
                .nombre.apply(lambda x: x.split(":")[0])
                .unique()
            ),
        )
    )
)

Sólo 20 de 126 entidades registradas subieron nuevos datos desde febrero de 2022

Cuándo aparecen estos datos?

In [104]:
datasets.groupby(
    [pd.Grouper(key="timestamp", freq="W"), "entidad"]
).nombre.count().reset_index().pivot_table(
    index="timestamp", columns="entidad", values="nombre"
).fillna(
    0
).astype(
    int
)

entidad,GobMunicipal,abc,abt,autoridadelectricidad,fondos,geoelectoral,inra,mapashistoricos,mddpyep,mddryt,mde,mdenerg,mdmaya,mono,nasa,onu,otros,prueba,sernap,universidades
timestamp,,,,,,,,,,,,,,,,,,,,
2022-03-20,166,0,15,0,2,8,4,4,1,3,1,0,1,1,9,2,1,2,2,4
2022-05-08,0,2,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2022-07-03,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0
2022-07-10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2022-07-31,0,0,0,0,0,0,0,0,0,0,0,11,0,0,0,0,0,0,0,0
2022-08-07,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0


_Por qué aparecen tantos datos en la misma semana de marzo?_ No parece algo exclusivo de una sóla entidad

In [134]:
display(
    Markdown(
        "{:.2%} de los datasets de `GobMunicipal` mencionan`Municipio de La Paz` en el título.".format(
            datasets[datasets.entidad == "GobMunicipal"]
            .titulo.str.contains("Municipio de La Paz")
            .value_counts(normalize=True)
            .to_dict()[True]
        )
    )
)

82.53% de los datasets de `GobMunicipal` mencionan`Municipio de La Paz` en el título.

Probablemente todos estos datasets hayan sidos subidos por el Gobierno Municipal de La Paz. Estos datasets no se concentran en ningún tema específico. A diferencia de la situación en otras entidades que suben actualizaciones, estos datos no sugieren ningún reporte específico en el Gobierno Municipal. Es como si hubieran decidido subir una carpeta llena de datos geográficos que tenían en el escritorio de un analista.

In [142]:
def show_entidad(entidad):
    if type(entidad) == str:
        return datasets[datasets.entidad == entidad][["timestamp", "titulo"]]
    elif type(entidad) == list:
        return datasets[datasets.entidad.isin(entidad)][
            ["timestamp", "entidad", "titulo"]
        ].sort_values('timestamp', ascending=False)


show_entidad("GobMunicipal")

,timestamp,titulo
57,2022-03-16 17:30:14,Mapa de Actividad Económica en el Municipio de La Paz
58,2022-03-16 17:30:14,Mapa de Contaminación Acústica en el Municipio de La Paz
59,2022-03-16 17:30:14,Mapa de Cobertura de Agua Potable en el Área Urbana del Municipio de La Paz
60,2022-03-16 17:30:14,Mapa de Servicio de Agua Potable en el Municipio de La Paz
61,2022-03-16 17:30:14,Mapa de Localización de Alojamientos y Albergues en el Municipio de La Paz
...,...,...
244,2022-03-16 17:30:14,Mapa de Vías Existentes en el Área Rural del Municipio de La Paz
245,2022-03-16 17:30:14,Mapa de Vias Metropolitanas en el Municipio de La Paz
246,2022-03-16 17:30:14,Mapa Vial del Municipio de La Paz
247,2022-03-16 17:30:14,Proyectos Modelo para el Área Rural del Municipio de La Paz


_Qué datos suben otras entidades?_

Algunas entidades han estado monitoreando focos de calor y deforestación

In [146]:
show_entidad(["nasa", "abt", "otros", "sernap"])

,timestamp,entidad,titulo
12,2022-07-07 00:38:53,otros,METOP_INPE_2022_MAY_JUN
14,2022-07-02 00:28:22,otros,"Monitoreo de Focos de Calor en Bolivia Satélite GOES de Mayo a Junio, INPE 2022"
15,2022-07-02 00:28:22,otros,"Monitoreo de Focos de Calor en Bolivia Satélite NOAA de Mayo a Junio, INPE 2022"
13,2022-07-02 00:28:22,otros,"Monitoreo de Focos de Calor en Bolivia Satélite AQUA de Mayo a Junio, INPE 2022"
38,2022-03-20 22:28:29,sernap,Deforestación y Regeneración en Bolivia 1990-2000
51,2022-03-20 22:28:29,nasa,Modelo Digital de Terreno (90m)
46,2022-03-20 22:28:29,nasa,Mapa de Luces de Bolivia 2012 - NASA
41,2022-03-20 22:28:29,otros,Cobertura de la Tierra en base al indice LAI Bolivia 2016
39,2022-03-20 22:28:29,sernap,Deforestación y Regeneración en Bolivia 2000-2010
55,2022-03-20 22:28:29,nasa,Hillshade (90m)


El Ministerio de Energía ha actualizado datos de estaciones y líneas de transmisión

In [143]:
show_entidad(["mdenerg", "autoridadelectricidad"])

,timestamp,entidad,titulo
0,2022-08-04 00:34:05,mdenerg,"Zonas y Área de Operación de Energía Eléctrica, Bolivia 2021"
2,2022-07-30 00:30:42,mdenerg,"Centrales Generadoras SIN 16 de Abril 2019, Bolivia"
3,2022-07-30 00:30:42,mdenerg,Centrales Generadoras SIN 3 de Marzo de 2021
4,2022-07-30 00:30:42,mdenerg,Líneas de Transmisión SIN Bolivia 16 de Abril de 2019
5,2022-07-30 00:30:42,mdenerg,Líneas de Transmisión SIN Bolivia 2 de Febrero de 2020
6,2022-07-30 00:30:42,mdenerg,Líneas de Transmisión SIN Bolivia 5 de Mayo de 2021
1,2022-07-30 00:30:42,mdenerg,"Centrales Generadoras SIN 12 de Febrero 2020, Bolivia"
10,2022-07-28 00:34:03,mdenerg,Subestaciones del SIN Febrero 2020
11,2022-07-28 00:34:03,mdenerg,Subestaciones del SIN Mayo 2021
8,2022-07-28 00:34:03,mdenerg,"Líneas de Media Tensión 2020, Bolivia"


El órgano electoral ha actualizado sus datos de recintos

In [82]:
show_entidad("geoelectoral")

,timestamp,titulo
141,2022-03-16 17:30:14,geoelectoral-departamento
142,2022-03-16 17:30:14,geoelectoral-departamentolz
143,2022-03-16 17:30:14,geoelectoral-dpa
144,2022-03-16 17:30:14,geoelectoral-municipio
145,2022-03-16 17:30:14,geoelectoral-municipiolz
146,2022-03-16 17:30:14,geoelectoral-provincia
147,2022-03-16 17:30:14,geoelectoral-provincialz
148,2022-03-16 17:30:14,geoelectoral-recinto


Alguna universidad ha trabajado en modelos de accesibilidad geográfica

In [85]:
show_entidad("universidades")

,timestamp,titulo
42,2022-03-20 22:28:29,Cobertura de la Tierra en base a LAI promedio de 8 días 21 septiembre de 2016
43,2022-03-20 22:28:29,Modelo de accesibilidad a comunidades (horas)
44,2022-03-20 22:28:29,Modelo de accesibilidad a capitales departamentales (horas)
45,2022-03-20 22:28:29,Modelo de accesibilidad a capitales de municipio (horas)


Han subido nuevos mapas históricos de Bolivia

In [86]:
show_entidad("mapashistoricos")

,timestamp,titulo
47,2022-03-20 22:28:29,Mapa historico de Bolivia 1856-b
48,2022-03-20 22:28:29,Mapa Historico de Bolivia 1834
49,2022-03-20 22:28:29,Mapa Historico de Bolivia 1856
50,2022-03-20 22:28:29,Mapa Historico de Bolvia 1865


También han actualizado algunos registros de titulaciones en el INRA

In [87]:
show_entidad("inra")

,timestamp,titulo
25,2022-03-20 22:28:29,Predios rurales titulados 2015
156,2022-03-16 17:30:14,Parcelas y predios titulados por el INRA 2016
210,2022-03-16 17:30:14,Propiedades Comunitarias tituladas por el INRA 2018
229,2022-03-16 17:30:14,Solar Campesino y Pequeña Propiedad titulados por el INRA 2018


Finalmente, otros datasets que aparecieron en GeoBolivia

In [151]:
show_entidad([e for e in datasets.entidad.unique() if e not in ["GobMunicipal", "nasa", "abt", "otros", "sernap", "mdenerg", "autoridadelectricidad", "geoelectoral", "universidades", "mapashistoricos", "inra"]])

,timestamp,entidad,titulo
21,2022-05-05 00:21:43,abc,"Puentes de la Red Vial Fundamental de Bolivia, 2021"
22,2022-05-05 00:21:43,abc,Red Vial Fundamental de Bolivia 2021
23,2022-03-20 22:28:29,prueba,Departamento
24,2022-03-20 22:28:29,mde,Establecimientos Educativos 2008 - cluster
26,2022-03-20 22:28:29,prueba,vias_La_Paz
27,2022-03-20 22:28:29,onu,"Indicador sobre tendencias de Cambio en Vegetación, 2017"
28,2022-03-20 22:28:29,mddryt,Cobertura y Uso Actual de la Tierra - 2010
33,2022-03-20 22:28:29,mddryt,"Mapa preliminar de Markas y Ayllus de las provincias: Ingavi, Pacajes y José Manuel Pando"
34,2022-03-20 22:28:29,mddpyep,Mapa de Potencialidades Productivas de Bolivia 2013
37,2022-03-20 22:28:29,mdmaya,"Mapa de Bosques de Bolivia, 2013"
